In [1]:
import pandas as pd

def get_semester_dates(year, semester):
    """
    Returns the start and end date of the semester based on the given year and semester type, respecting UTC time.
    Semesters:
    - Spring: January to June
    - Summer: June to July
    - Winter: August to December
    """
    if semester == 'Spring':
        return pd.Timestamp(f'{year}-01-01', tz='UTC'), pd.Timestamp(f'{year}-06-30', tz='UTC')
    elif semester == 'Summer':
        return pd.Timestamp(f'{year}-06-01', tz='UTC'), pd.Timestamp(f'{year}-07-31', tz='UTC')
    elif semester == 'Winter':
        return pd.Timestamp(f'{year}-08-01', tz='UTC'), pd.Timestamp(f'{year}-12-31', tz='UTC')
    else:
        raise ValueError("Invalid semester. Choose between 'Spring', 'Summer', or 'Winter'.")


In [57]:
import pandas as pd 
from utils.constants import ENROLLMENTS_PATH

enrollments_df = pd.read_csv(ENROLLMENTS_PATH)
modules_df = pd.read_csv("")
module_items_df = pd.read_csv("")


year = 2024
semester = "Spring"


def calculate_modules_per_student(enrollments_df, modules_df, module_items_df):
  """
  Calcula el número promedio de módulos completados por estudiante.

  Args:
    enrollments_df: DataFrame con información de las matrículas.
    modules_df: DataFrame con información de los módulos.
    module_items_df: DataFrame con información de los elementos de los módulos.

  Returns:
    Un DataFrame con el ID del estudiante y el número de módulos completados.
  """

  # Fusionar los DataFrames para obtener la relación entre estudiantes, módulos y elementos
  df_merged = enrollments_df.merge(modules_df, on='course_id')
  df_merged = df_merged.merge(module_items_df, on='module_id')

  # Agrupar por estudiante y contar los módulos únicos completados
  modules_per_student = df_merged.groupby('user_id')['module_id'].nunique()

  return modules_per_student

# Ejemplo de uso (reemplaza con tus datos reales)
enrollments = pd.read_csv('enrollments.csv')
modules = pd.read_csv('modules.csv')
module_items = pd.read_csv('module_items.csv')

result = calculate_modules_per_student(enrollments, modules, module_items)
print(result)

In [150]:
import pandas as pd 
from utils.constants import ENROLLMENTS_PATH, SCORES_PATH, COURSES_PATH

year = 2024
semester = "Spring"
enrollments_df = pd.read_csv(ENROLLMENTS_PATH)
scores_df = pd.read_csv(SCORES_PATH)
courses_df = pd.read_csv(COURSES_PATH)

def courses_with_high_failing_enrollments(
    enrollments_df,
    scores_df,
    year,
    semester,
    failing_threshold=60,
    fail_count_threshold=10  # Adjust based on your criteria
):
    # Convert dates to datetime
    enrollments_df['value.created_at'] = pd.to_datetime(enrollments_df['value.created_at'], utc=True)
    scores_df['value.created_at'] = pd.to_datetime(scores_df['value.created_at'], utc=True)
    

    # Get semester dates
    start_date, end_date = get_semester_dates(year, semester)
    semester_start = pd.to_datetime(start_date)
    semester_end = pd.to_datetime(end_date)
    
    # Filter enrollments during the semester
    semester_enrollments = enrollments_df[
        (enrollments_df['value.created_at'] >= semester_start) &
        (enrollments_df['value.created_at'] <= semester_end) &
        (enrollments_df['value.workflow_state'] == 'available')
    ]
    # Merge enrollments with scores
    merged_df = semester_enrollments.merge(
        scores_df[['value.enrollment_id', 'value.final_score']],
        left_on='key.id',  # Enrollment ID in enrollments_df
        right_on='value.enrollment_id',  # Enrollment ID in scores_df
        how='left'
    )

    # Identify failing enrollments
    failing_enrollments = merged_df[
        (merged_df['value.final_score'] < failing_threshold) |
        (merged_df['value.final_score'].isnull())  # Assuming null scores are failing
    ]

    # Count failing enrollments per course
    failing_counts = failing_enrollments.groupby('value.enrollment_id').size().reset_index(name='failing_enrollments')
    
    # Identify courses exceeding the failing enrollment threshold
    flagged_courses = failing_counts[
        failing_counts['failing_enrollments'] >= fail_count_threshold
    ]
    
    # Optionally, merge with course names if available
    # If you have a courses DataFrame with 'id' and 'name' columns
    # flagged_courses = flagged_courses.merge(
    #     courses_df[['key.id', 'value.name']],
    #     left_on='value.course_id',
    #     right_on='key.id',
    #     how='left'
    # )
    
    return flagged_courses

In [151]:
year = 2024
semester = 'Spring'
failing_threshold = 60
courses_failed_threshold = 2
failing_threshold = 60  # Adjust as per your institution's passing grade
fail_count_threshold = 10

flagged_courses_df = courses_with_high_failing_enrollments(
    enrollments_df,
    scores_df,
    year,
    semester,
    failing_threshold,
    fail_count_threshold,
)

print(flagged_courses_df)

    value.enrollment_id  failing_enrollments
7               12014.0                   17
11              12023.0                   16
14              12027.0                   19
17              12033.0                   19
20              12036.0                   16
21              12037.0                   27
24              12044.0                   18
31              12061.0                   17
34              12064.0                   18
41              12074.0                   18
71              12173.0                   10


In [179]:
def create_student_retention_rate_per_course(enrollments_df, year, semester):
    """
    Calculates the student retention rate per course for a given semester.

    Returns a DataFrame with course IDs and their corresponding retention rates.
    """
    # Ensure necessary columns are present
    required_columns = [
        'value.course_id', 'value.created_at', 'value.updated_at', 
        'value.workflow_state'
    ]
    for col in required_columns:
        if col not in enrollments_df.columns:
            raise ValueError(f"Column '{col}' is missing from enrollments_df.")

    # Get semester start and end dates
    start_semester, end_semester = get_semester_dates(year, semester)

    # Convert date columns to datetime
    enrollments_df['created_at'] = pd.to_datetime(enrollments_df['value.created_at'], utc=True)
    enrollments_df['updated_at'] = pd.to_datetime(enrollments_df['value.updated_at'], utc=True)

    # Filter enrollments within the semester
    semester_enrollments = enrollments_df[
        (enrollments_df['created_at'] <= end_semester) & 
        (enrollments_df['updated_at'] >= start_semester)
    ]

    # Initialize a list to store retention data
    retention_data = []

    # Group by course_id
    for course_id, group in semester_enrollments.groupby('value.course_id'):
        # Initial Enrollment: Students active at the start of the semester
        initial_enrollment = group[
            (group['created_at'] <= start_semester) & 
            (group['value.workflow_state'] == 'active')
        ].shape[0]

        # Final Enrollment: Students active at the end of the semester
        final_enrollment = group[
            (group['updated_at'] >= end_semester) & 
            (group['value.workflow_state'] == 'active')
        ].shape[0]

        # Calculate retention rate
        if initial_enrollment > 0:
            retention_rate = (final_enrollment / initial_enrollment) * 100
        else:
            retention_rate = 0

        # Append to the retention data list
        retention_data.append({
            'course_id': str(course_id),
            'retention_rate': retention_rate
        })

    # Create a DataFrame from the retention data
    retention_df = pd.DataFrame(retention_data)

    return retention_df


In [222]:
import pandas as pd 
from utils.constants import ASSIGNMENTS_PATH, SUBMISSIONS_PATH

year = 2024
semester = "Spring"
assignments_df = pd.read_csv(ASSIGNMENTS_PATH)
submissions_df = pd.read_csv(SUBMISSIONS_PATH)

def calculate_course_completion_rate(assignments_df, submissions_df, year, semester):
    start_semester, end_semester = get_semester_dates(year, semester)
    assignments_df['value.created_at'] = pd.to_datetime(assignments_df['value.created_at'], errors='coerce', utc=True)
    assignments_df['value.due_at'] = pd.to_datetime(assignments_df['value.due_at'], errors='coerce', utc=True)
    submissions_df['value.submitted_at'] = pd.to_datetime(submissions_df['value.submitted_at'], errors='coerce', utc=True)

    # Step 3: Filter Assignments within the Semester
    # We'll consider assignments that are due within the semester
    assignments_df = assignments_df[
        (assignments_df['value.due_at'] >= start_semester) &
        (assignments_df['value.due_at'] <= end_semester)
    ]

    # Merge the DataFrames to get student-assignment-submission relationships
    merged_df = pd.merge(
        submissions_df,
        assignments_df,
        left_on='value.assignment_id',
        right_on='key.id',
        how='inner'
    )

    # Calculate the completion rate for each course
    completion_rates = merged_df.groupby("value.course_id")#.apply(lambda x: (x["value.score_y"].count() / len(x)) * 100)
    print("'value.course_id' in merged_df.columns:", 'value.course_id' in merged_df.columns)


    # mask_1 = completion_rates["value.score"] > 0.0
    return completion_rates

/var/folders/cj/p3yg45h13dv_1n4x8d34dy300000gn/T/ipykernel_58578/1916203005.py:6: DtypeWarning: Columns (55,56,58,59,60,61,62,63,64) have mixed types. Specify dtype option on import or set low_memory=False.
  assignments_df = pd.read_csv(ASSIGNMENTS_PATH)
/var/folders/cj/p3yg45h13dv_1n4x8d34dy300000gn/T/ipykernel_58578/1916203005.py:7: DtypeWarning: Columns (8,9,10,11,15,19,20,21,22,24,25,30,32,33,34,37,39,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  submissions_df = pd.read_csv(SUBMISSIONS_PATH)


In [223]:
completion_rates = calculate_course_completion_rate(assignments_df, submissions_df, year, semester)
completion_rates.describe()

'value.course_id' in merged_df.columns: True


key.id_x                                                     \
                   count          mean           std        min         25%   
value.course_id                                                               
3203                 3.0  3.542925e+06      1.000000  3542924.0  3542924.50   
7977                 1.0  3.548737e+06           NaN  3548737.0  3548737.00   
10786                2.0  3.575792e+06      0.707107  3575792.0  3575792.25   
11041               53.0  3.546969e+06     15.443445  3546943.0  3546956.00   
11042               44.0  3.521500e+06     12.845233  3521478.0  3521488.75   
...                  ...           ...           ...        ...         ...   
13258               95.0  3.646067e+06  10923.412276  3642243.0  3642334.50   
13265              135.0  3.625165e+06  16786.952867  3612053.0  3618078.50   
13318               52.0  3.610033e+06  16102.324255  3585710.0  3611487.75   
13389              648.0  3.660324e+06   1079.334598  3659187.0  3659671.75   
13427               54.0  3.692980e+06     15.732133  3692953.0  3692966.25   

                                                  value.attachment_id       \
                       50%         75%        max               count mean   
value.course_id                                                              
3203             3542925.0  3542925.50  3542926.0                 0.0  NaN   
7977             3548737.0  3548737.00  3548737.0                 0.0  NaN   
10786            3575792.5  3575792.75  3575793.0                 0.0  NaN   
11041            3546969.0  3546982.00  3546995.0                 0.0  NaN   
11042            3521499.5  3521510.25  3521521.0                 0.0  NaN   
...                    ...         ...        ...                 ...  ...   
13258            3642375.0  3642415.50  3680737.0                 0.0  NaN   
13265            3619047.0  3619234.50  3680720.0                 0.0  NaN   
13318            3611500.5  3611660.25  3674833.0                 0.0  NaN   
13389            3660404.5  3660848.25  3673207.0                 0.0  NaN   
13427            3692979.5  3692992.75  3693006.0                 0.0  NaN   

                 ... value.position        \
                 ...            75%   max   
value.course_id  ...                        
3203             ...           39.0  39.0   
7977             ...            4.0   4.0   
10786            ...            5.0   5.0   
11041            ...            1.0   1.0   
11042            ...            2.0   2.0   
...              ...            ...   ...   
13258            ...           17.0  24.0   
13265            ...           97.0  99.0   
13318            ...            5.0   5.0   
13389            ...            6.0   6.0   
13427            ...            5.0   6.0   

                value.turnitin_settings.exclude_small_matches_value           \
                                                              count mean std   
value.course_id                                                                
3203                                                           0.0   NaN NaN   
7977                                                           0.0   NaN NaN   
10786                                                          0.0   NaN NaN   
11041                                                          0.0   NaN NaN   
11042                                                          0.0   NaN NaN   
...                                                            ...   ...  ..   
13258                                                          0.0   NaN NaN   
13265                                                          0.0   NaN NaN   
13318                                                          0.0   NaN NaN   
13389                                                          0.0   NaN NaN   
13427                                                          0.0   NaN NaN   

                                     
                min 25% 50

In [189]:
def calculate_course_completion_rate(assignments_df, submissions_df, year, semester):

    start_semester, end_semester = get_semester_dates(year, semester)

    # Step 2: Convert Date Columns to Datetime
    assignments_df['value.created_at'] = pd.to_datetime(assignments_df['value.created_at'], errors='coerce', utc=True)
    assignments_df['value.due_at'] = pd.to_datetime(assignments_df['value.due_at'], errors='coerce', utc=True)
    submissions_df['value.submitted_at'] = pd.to_datetime(submissions_df['value.submitted_at'], errors='coerce', utc=True)

    # Step 3: Filter Assignments within the Semester
    # We'll consider assignments that are due within the semester
    assignments_in_semester = assignments_df[
        (assignments_df['value.due_at'] >= start_semester) &
        (assignments_df['value.due_at'] <= end_semester)
    ]

    # Step 4: Filter Submissions that correspond to these assignments
    submissions_in_semester = submissions_df[
        submissions_df['value.assignment_id'].isin(assignments_in_semester['key.id'])
    ]

    # Step 5: Merge the DataFrames to get student-assignment-submission relationships
    merged_df = pd.merge(
        submissions_in_semester,
        assignments_in_semester,
        left_on='value.assignment_id',
        right_on='key.id',
        how='inner'
    )

    # Step 6: Ensure 'value.course_id' is available
    if 'value.course_id' not in assignments_in_semester.columns:
        if 'value.context_id' in assignments_in_semester.columns:
            # Assuming 'value.context_id' represents 'course_id'
            assignments_in_semester['value.course_id'] = assignments_in_semester['value.context_id']
        else:
            raise ValueError("Column 'value.course_id' or 'value.context_id' is missing from assignments_df.")

    # Step 7: Calculate the completion rate for each course
    # Total expected submissions per course: number of assignments * number of students enrolled
    # For simplification, we'll assume the total expected submissions equal the number of assignments
    total_assignments_per_course = assignments_in_semester.groupby('value.course_id')['key.id'].nunique().reset_index()
    total_assignments_per_course.rename(columns={'key.id': 'total_assignments'}, inplace=True)

    # Actual submissions per course
    submissions_per_course = merged_df.groupby('value.course_id')['value.submission_type'].count().reset_index()
    submissions_per_course.rename(columns={'value.submission_type': 'total_submissions'}, inplace=True)

    # Merge the totals
    completion_df = pd.merge(
        total_assignments_per_course,
        submissions_per_course,
        on='value.course_id',
        how='left'
    )

    # Fill NaN values in total_submissions with 0
    completion_df['total_submissions'] = completion_df['total_submissions'].fillna(0)

    # Calculate completion rate per course
    completion_df['completion_rate'] = (completion_df['total_submissions'] / completion_df['total_assignments']) * 100

    # Convert course_id to string for consistency
    completion_df['value.course_id'] = completion_df['value.course_id'].astype(str)

    # Return the DataFrame with course IDs and completion rates
    return completion_df[['value.course_id', 'completion_rate']]


In [190]:
completion_df = calculate_course_completion_rate(assignments_df, submissions_df, year, semester)

/var/folders/cj/p3yg45h13dv_1n4x8d34dy300000gn/T/ipykernel_58578/941270166.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  assignments_in_semester['value.course_id'] = assignments_in_semester['value.context_id']


In [192]:
completion_df.describe()

,completion_rate
count,737.000000
mean,219.751555
std,474.159437
min,0.000000
25%,0.000000
50%,0.000000
75%,242.857143
max,3900.000000
